In [3]:
from datamodel import (
    Listing,
    Observation,
    Order,
    OrderDepth,
    ProsperityEncoder,
    UserId,
    Symbol,
    Trade,
    TradingState,
)
from typing import List
import copy
import numpy as np
import math
from itertools import permutations
import Round_2

###############
import sys
import os

sys.path.append(os.path.abspath("../"))

import webbrowser
from datetime import datetime
from pathlib import Path
from typing import Any, Optional
from importlib import reload
from collections import defaultdict
from functools import partial, reduce
from http.server import HTTPServer, SimpleHTTPRequestHandler

from prosperity3bt.data import has_day_data, read_day_data
from prosperity3bt.file_reader import (
    FileReader,
    FileSystemReader,
    PackageResourcesReader,
)
from prosperity3bt.models import BacktestResult
from prosperity3bt.runner import run_backtest
################


import json
from typing import Any

In [4]:
class tradable_product:
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    KELP = "KELP"
    SQUID_INK = "SQUID_INK"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    PICNIC_BASKET1 = "PICNIC_BASKET1"
    PICNIC_BASKET2 = "PICNIC_BASKET2"
    DJEMBES = "DJEMBES"


class position_limit:
    KELP = 50
    RAINFOREST_RESIN = 50
    SQUID_INK = 50
    CROISSANTS = 250
    JAMS = 350
    PICNIC_BASKET1 = 60
    PICNIC_BASKET2 = 100
    DJEMBES = 60

In [5]:
def parse_data(data_root: Optional[Path]) -> FileReader:
    if data_root is not None:
        return FileSystemReader(data_root)
    else:
        return PackageResourcesReader()


def parse_days(file_reader: FileReader, days: list[str]) -> list[tuple[int, int]]:
    parsed_days = []

    for arg in days:
        if "-" in arg:
            round_num, day_num = map(int, arg.split("-", 1))

            if not has_day_data(file_reader, round_num, day_num):
                print(f"Warning: no data found for round {round_num} day {day_num}")
                continue

            parsed_days.append((round_num, day_num))
        else:
            round_num = int(arg)

            parsed_days_in_round = []
            for day_num in range(-5, 6):
                if has_day_data(file_reader, round_num, day_num):
                    parsed_days_in_round.append((round_num, day_num))

            if len(parsed_days_in_round) == 0:
                print(f"Warning: no data found for round {round_num}")
                continue

            parsed_days.extend(parsed_days_in_round)

    if len(parsed_days) == 0:
        print("Error: did not find data for any requested round/day")
        sys.exit(1)

    return parsed_days


def parse_out(out: Optional[Path], no_out: bool) -> Optional[Path]:
    if out is not None:
        return out

    if no_out:
        return None

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return Path.cwd() / "backtests" / f"{timestamp}.log"


def print_day_summary(result: BacktestResult) -> None:
    last_timestamp = result.activity_logs[-1].timestamp

    product_lines = []
    total_profit = 0

    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        product = row.columns[2]
        profit = row.columns[-1]

        product_lines.append(f"{product}: {profit:,.0f}")
        total_profit += profit

    print(*reversed(product_lines), sep="\n")
    print(f"Total profit: {total_profit:,.0f}")


def merge_results(
    a: BacktestResult,
    b: BacktestResult,
    merge_profit_loss: bool,
    merge_timestamps: bool,
) -> BacktestResult:
    sandbox_logs = a.sandbox_logs[:]
    activity_logs = a.activity_logs[:]
    trades = a.trades[:]

    if merge_timestamps:
        a_last_timestamp = a.activity_logs[-1].timestamp
        timestamp_offset = a_last_timestamp + 100
    else:
        timestamp_offset = 0

    sandbox_logs.extend([row.with_offset(timestamp_offset) for row in b.sandbox_logs])
    trades.extend([row.with_offset(timestamp_offset) for row in b.trades])

    if merge_profit_loss:
        profit_loss_offsets = defaultdict(float)
        for row in reversed(a.activity_logs):
            if row.timestamp != a_last_timestamp:
                break

            profit_loss_offsets[row.columns[2]] = row.columns[-1]

        activity_logs.extend(
            [
                row.with_offset(timestamp_offset, profit_loss_offsets[row.columns[2]])
                for row in b.activity_logs
            ]
        )
    else:
        activity_logs.extend(
            [row.with_offset(timestamp_offset, 0) for row in b.activity_logs]
        )

    return BacktestResult(a.round_num, a.day_num, sandbox_logs, activity_logs, trades)


def write_output(output_file: Path, merged_results: BacktestResult) -> None:
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with output_file.open("w+", encoding="utf-8") as file:
        file.write("Sandbox logs:\n")
        for row in merged_results.sandbox_logs:
            file.write(str(row))

        file.write("\n\n\nActivities log:\n")
        file.write(
            "day;timestamp;product;bid_price_1;bid_volume_1;bid_price_2;bid_volume_2;bid_price_3;bid_volume_3;ask_price_1;ask_volume_1;ask_price_2;ask_volume_2;ask_price_3;ask_volume_3;mid_price;profit_and_loss\n"
        )
        file.write("\n".join(map(str, merged_results.activity_logs)))

        file.write("\n\n\n\n\nTrade History:\n")
        file.write("[\n")
        file.write(",\n".join(map(str, merged_results.trades)))
        file.write("]")


def print_overall_summary(results: list[BacktestResult]) -> None:
    print("Profit summary:")

    total_profit = 0
    for result in results:
        last_timestamp = result.activity_logs[-1].timestamp

        profit = 0
        for row in reversed(result.activity_logs):
            if row.timestamp != last_timestamp:
                break

            profit += row.columns[-1]

        print(f"Round {result.round_num} day {result.day_num}: {profit:,.0f}")
        total_profit += profit

    print(f"Total profit: {total_profit:,.0f}")

    return total_profit


def format_path(path: Path) -> str:
    cwd = Path.cwd()
    if path.is_relative_to(cwd):
        return str(path.relative_to(cwd))
    else:
        return str(path)


class HTTPRequestHandler(SimpleHTTPRequestHandler):
    def do_GET(self):
        self.server.shutdown_flag = True
        return super().do_GET()

    def end_headers(self) -> None:
        self.send_header("Access-Control-Allow-Origin", "*")
        return super().end_headers()

    def log_message(self, format: str, *args: Any) -> None:
        return


class CustomHTTPServer(HTTPServer):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.shutdown_flag = False


def open_visualizer(output_file: Path) -> None:
    http_handler = partial(HTTPRequestHandler, directory=str(output_file.parent))
    http_server = CustomHTTPServer(("localhost", 0), http_handler)

    webbrowser.open(
        f"https://jmerle.github.io/imc-prosperity-3-visualizer/?open=http://localhost:{http_server.server_port}/{output_file.name}"
    )

    while not http_server.shutdown_flag:
        http_server.handle_request()

In [6]:
all_data = []
file_reader = parse_data(
    Path(
        "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/prosperity3bt/resources"
    )
)
days = parse_days(file_reader, [str(day) for day in range(2, 3)])
for round_num, day_num in days:
    all_data.append(read_day_data(file_reader, round_num, day_num, no_names=True))

In [7]:
def main(all_data, params: dict, output_file=None) -> None:
    results = []
    file_reader = parse_data(
        Path(
            "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/prosperity3bt/resources"
        )
    )
    days = parse_days(file_reader, [str(day) for day in range(2, 3)])
    for i in range(len(days)):
        round_num, day_num = days[i]
        reload(Round_2)
        data = all_data[i]
        result = run_backtest(
            trader=Round_2.Trader(params),
            data=data,
            file_reader=file_reader,
            round_num=round_num,
            day_num=day_num,
            print_output=False,
            trade_matching_mode="worse",
            no_names=True,  # args.no_names,
            show_progress_bar=False,
        )

        # print_day_summary(result)
        if len(days) > 1:
            print()

        results.append(result)

    if output_file is not None:
        merged_results = reduce(
            lambda a, b: merge_results(
                a, b, merge_profit_loss=True, merge_timestamps=True
            ),
            results,
        )
        write_output(output_file, merged_results)
        print(f"\nSuccessfully saved backtest results to {format_path(output_file)}")
        open_visualizer(output_file)

    final_profit = print_overall_summary(results)
    return final_profit

In [ ]:
params = {
    tradable_product.RAINFOREST_RESIN: {
        "ask_slip": 1,
        "bid_slip": -1,
        "decay": 0.1,
        "threshold": 0.0,
    },
    tradable_product.KELP: {
        "alpha": np.array([[-5.10651298e-04], [9.10917102e-06]]),
        "beta": np.array([[1.0, -0.93940982]]),
        "gamma": np.array([[-0.09555001, -0.27741155], [-0.00124705, -0.47838647]]),
        "decay": 0.6,
        "threshold": 0.8,
    },
    tradable_product.SQUID_INK: {
        "z_threshold": 1.7,
    },
    tradable_product.PICNIC_BASKET1: {
        tradable_product.CROISSANTS: 6,
        tradable_product.JAMS: 3,
        tradable_product.DJEMBES: 1,
        "alpha": np.array([[-0.00053585], [0.00605571]]),
        "beta": np.array([[1.0, -0.07278752]]),
        "gamma": np.array([[-0.14600432, 0.00577994], [0.41390133, -0.05275109]]),
        "z_threshold": 2.4,
        "z_threshold2": 2.6,
    },
    tradable_product.PICNIC_BASKET2: {
        tradable_product.CROISSANTS: 4,
        tradable_product.JAMS: 2,
        tradable_product.DJEMBES: 0,
        "alpha": np.array([[-0.00019455], [0.00026457]]),
        "beta": np.array([[1.0, -1.94279816]]),
        "gamma": np.array([[-0.03700845, -0.01196759], [-0.01018269, -0.0202882]]),
        "z_threshold": 2.4,
        "z_threshold2": 1.2,
    },
    tradable_product.CROISSANTS: {
        "alpha": np.array([[-0.00053585], [0.00605571]]),
        "beta": np.array([[1.0, -0.07278752]]),
        "gamma": np.array([[-0.14600432, 0.00577994], [0.41390133, -0.05275109]]),
        "z_threshold": 3.8,
    },
    tradable_product.JAMS: {
        "alpha": np.array([[-0.00020707], [0.00051619]]),
        "beta": np.array([[1.0, -0.11137341]]),
        "gamma": np.array([[-0.06666785, 0.00483818], [0.26798214, -0.04917357]]),
        "z_threshold": 5.8,
    },
    tradable_product.DJEMBES: {
        "alpha": np.array([[-8.64219383e-05], [5.49907373e-04]]),
        "beta": np.array([[1.0, -0.2278196]]),
        "gamma": np.array([[0.01386504, 0.00224211], [0.00333236, -0.04007833]]),
        "z_threshold": 3,
    },
}


result = main(
    all_data=all_data,
    params=params,
    output_file=Path(
        "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/Round 2/test.log"
    ),
)

c:\Users\edmun\OneDrive\Desktop\2025-IMC-Global-Trading-Challenge\Round 2\Round_2.py:414: RuntimeWarning: Mean of empty slice.
  return (value - sample.mean()) / sample.std()
c:\Users\edmun\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\edmun\anaconda3\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\edmun\anaconda3\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\edmun\anaconda3\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\edmun\OneDrive\Desktop\2025-IMC-Global-Trading-Challenge\Round 2\Round_2.py:414: RuntimeWarning: invalid value encountered i





Successfully saved backtest results to test.log
Profit summary:
Round 2 day -1: 119,258
Round 2 day 0: 134,975
Round 2 day 1: 101,010
Total profit: 355,243


In [13]:
# param_range = np.arange(-1, 1, 0.25)
# all_combi = list(permutations(param_range, 2))

best = 0
perm = {"PNL": [], "i": [], "j": []}

print(f"Permuations Count: {len(list(permutations(np.arange(1, 3, 0.2), 2)))}")

for i in np.arange(1.7, 4, 0.1):
    j = 3
    window = 0.3
    threshold = 0
    params = {
        tradable_product.RAINFOREST_RESIN: {
            "ask_slip": 1,
            "bid_slip": -1,
            "decay": 0.1,
            "threshold": 0.0,
        },
        tradable_product.KELP: {
            "alpha": np.array([[-5.10651298e-04], [9.10917102e-06]]),
            "beta": np.array([[1.0, -0.93940982]]),
            "gamma": np.array([[-0.09555001, -0.27741155], [-0.00124705, -0.47838647]]),
            "decay": 0.6,
            "threshold": 0.8,
        },
        tradable_product.SQUID_INK: {
            "z_threshold": i,
        },
        tradable_product.PICNIC_BASKET1: {
            tradable_product.CROISSANTS: 6,
            tradable_product.JAMS: 3,
            tradable_product.DJEMBES: 1,
            "alpha": np.array([[-0.00053585], [0.00605571]]),
            "beta": np.array([[1.0, -0.07278752]]),
            "gamma": np.array([[-0.14600432, 0.00577994], [0.41390133, -0.05275109]]),
            "z_threshold": i,
            "z_threshold2": 3,
        },
        tradable_product.PICNIC_BASKET2: {
            tradable_product.CROISSANTS: 4,
            tradable_product.JAMS: 2,
            tradable_product.DJEMBES: 0,
            "alpha": np.array([[-0.00019455], [0.00026457]]),
            "beta": np.array([[1.0, -1.94279816]]),
            "gamma": np.array([[-0.03700845, -0.01196759], [-0.01018269, -0.0202882]]),
            "z_threshold": 2.4,
            "z_threshold2": 1.2,
        },
        tradable_product.CROISSANTS: {
            "alpha": np.array([[-0.00053585], [0.00605571]]),
            "beta": np.array([[1.0, -0.07278752]]),
            "gamma": np.array([[-0.14600432, 0.00577994], [0.41390133, -0.05275109]]),
            "z_threshold": 3.8,
        },
        tradable_product.JAMS: {
            "alpha": np.array([[-0.00020707], [0.00051619]]),
            "beta": np.array([[1.0, -0.11137341]]),
            "gamma": np.array([[-0.06666785, 0.00483818], [0.26798214, -0.04917357]]),
            "z_threshold": 5.8,
        },
        tradable_product.DJEMBES: {
            "alpha": np.array([[-8.64219383e-05], [5.49907373e-04]]),
            "beta": np.array([[1.0, -0.2278196]]),
            "gamma": np.array([[0.01386504, 0.00224211], [0.00333236, -0.04007833]]),
            "z_threshold": i,
        },
    }

    result = main(all_data=all_data, params=params)

    perm["PNL"].append(result)
    perm["i"].append(i)
    perm["j"].append(j)

    if result > best:
        best = result
        print(best, i, j)

print(best)


Permuations Count: 90


c:\Users\edmun\OneDrive\Desktop\2025-IMC-Global-Trading-Challenge\Round 2\Round_2.py:414: RuntimeWarning: Mean of empty slice.
  return (value - sample.mean()) / sample.std()
c:\Users\edmun\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\edmun\anaconda3\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\edmun\anaconda3\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\edmun\anaconda3\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\edmun\OneDrive\Desktop\2025-IMC-Global-Trading-Challenge\Round 2\Round_2.py:414: RuntimeWarning: invalid value encountered i




Profit summary:
Round 2 day -1: 1,712
Round 2 day 0: 2,379
Round 2 day 1: 6,813
Total profit: 10,904
10904.0 1.7 3



Profit summary:
Round 2 day -1: 2,118
Round 2 day 0: 2,715
Round 2 day 1: 5,234
Total profit: 10,067



Profit summary:
Round 2 day -1: 2,007
Round 2 day 0: 2,150
Round 2 day 1: 4,417
Total profit: 8,574



Profit summary:
Round 2 day -1: 3,399
Round 2 day 0: 584
Round 2 day 1: 3,547
Total profit: 7,530



Profit summary:
Round 2 day -1: 3,558
Round 2 day 0: -971
Round 2 day 1: 4,736
Total profit: 7,323



Profit summary:
Round 2 day -1: 3,318
Round 2 day 0: -1,473
Round 2 day 1: 5,747
Total profit: 7,592



Profit summary:
Round 2 day -1: 3,687
Round 2 day 0: -2,376
Round 2 day 1: 4,727
Total profit: 6,038



Profit summary:
Round 2 day -1: 3,667
Round 2 day 0: -3,265
Round 2 day 1: 2,994
Total profit: 3,396



Profit summary:
Round 2 day -1: 2,873
Round 2 day 0: -3,870
Round 2 day 1: 2,077
Total profit: 1,080



Profit summary:
Round 2 day -1: 2,069
Round 2 day 0: 

In [14]:
import pandas as pd

pd.DataFrame.from_dict(perm).sort_values(by="PNL", ascending=False)

,PNL,i,j
0,10904.0,1.7,3
1,10067.0,1.8,3
2,8574.0,1.9,3
5,7592.0,2.2,3
3,7530.5,2.0,3
4,7323.0,2.1,3
6,6037.5,2.3,3
7,3396.0,2.4,3
8,1080.0,2.5,3
22,667.5,3.9,3
